<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Baseline" data-toc-modified-id="Baseline-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Baseline</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Подключим необходимые библиотеки

In [16]:
import pandas as pd
from pymystem3 import Mystem
import re
import nltk
nltk.download('wordnet')
import spacy
import sklearn.metrics as metrics
import torch
import transformers  # pytorch transformers
import numpy as np
import sklearn as sk

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
import warnings


warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Переключимся на CPU

In [17]:
device = torch.device('cpu')

Для лемматизации загрузим пак английских слов

In [18]:
disabled_pipes = [ "parser",  "ner"]
sp = spacy.load('C://Users/User/.conda/envs/stack-overflow/Lib/site-packages/en_core_web_sm/en_core_web_sm-3.4.0',disable=disabled_pipes)

Прочитаем данные

In [20]:
data = pd.read_csv('D://toxic_comments.csv')
df = data.copy()
df = df.sample(70_000)

Удалим дубликаты

In [23]:
df = df.drop_duplicates()

С помощью регулярных выражений оставим только английские слова, символы, знаки препинания и цифры удалим

In [24]:
df['re'] = df['text'].apply(lambda x : " ".join((re.sub(r'(?i)[^a-z]+', ' ', x).split())))

Проводим лемматизацию

In [25]:
%%time
df['lemm'] = df['re'].apply(lambda x : " ".join(token.lemma_ for token in sp(x)))

CPU times: total: 4min 59s
Wall time: 4min 59s


Проверям на пропущенные значения

In [26]:
len(df['lemm'])-len(df['lemm'].drop_duplicates())

336

Удаляем пропущенные значения

In [27]:
df = df.drop_duplicates()
df = df.dropna()

Теперь удалим ранее созданный столбец 're' который мы создали для предложений после использования регулярных выражений, изначальный столбец 'text', он нам не понадобится и 'Unnamed: 0 ' потому что он копирует id

In [28]:
df = df.drop(['re','text','Unnamed: 0'],axis = 1 )

Разделим данные на обучающую и тестовую выборку

In [29]:
X = df.drop(['toxic'],axis = 1 )
y = df['toxic']
X_train,X_test,y_train,y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

Приведём к нужному типу

In [30]:
X_train = X_train.values
X_test = X_test.values

## Обучение

### Baseline

Создадим пайплайн в котором проверим базовые модели: наивный байес, метод опорных векторов, логистическая регрессия и метод К-ближайших соседей. В нём закодируем текст с помощью TF-IDF векторизатора, далее проверим каждую модель и выберем лучшую и на ней вычислим f1 

In [31]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stop_words)


pipe = Pipeline(steps = [('tf_idf_vectorization',TfidfVectorizer(stop_words=stop_words)),('clf',MultinomialNB())])

search_space = [{'clf': [MultinomialNB()]},
                {'clf' :[LinearSVC()]},
                {'clf':[LogisticRegression(C = 10)],
                'clf__solver': ['liblinear']},
                {'clf':[KNeighborsClassifier()],
                'clf__n_neighbors':[5,6]}]
             

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
#scoring={F1-мера':metrics.make_scorer(metrics.f1_score)}

grid = GridSearchCV(estimator = pipe, param_grid = search_space, cv=3,
                   scoring = metrics.make_scorer(metrics.f1_score), return_train_score = True, n_jobs = -1, refit=True)

In [33]:
%%time
best_model = grid.fit(X_train.ravel(),y_train.ravel())

CPU times: total: 3.25 s
Wall time: 1min 44s


In [34]:
print(best_model.best_score_,best_model.best_params_)

0.7474833593485416 {'clf': LinearSVC()}


Так как я заметил, что очень большая разница в F1-score на тестовых и тренировочных данных я решил тут же проверить на тестовой выборке, я знаю так делать нельзя, но не знаю как сделать по-другому.

In [35]:
pred = best_model.predict(X_test.ravel())
print(f1_score(y_test, pred))

0.7769953051643192


Точность нас  устраивает, но попробуем её улучшить,попробуем градиентный бустинг

### CatBoost

С помощью TF-IDF создадим из предложений вектора на основе которых будем обучаться

In [36]:
X_train = count_tf_idf.fit_transform(X_train.ravel())
X_test = count_tf_idf.transform(X_test.ravel())

In [37]:
%%time
model = CatBoostClassifier(
    iterations=1500,
    depth = 6,
    random_seed=43,
    loss_function = 'Logloss'
)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=False,
)

CPU times: total: 1h 17s
Wall time: 8min 55s


Проверяем на тренировочных данных

In [38]:
%%time
y_pred = model.predict(X_train)
f1_score(y_train, y_pred)

CPU times: total: 2.22 s
Wall time: 370 ms


0.8157324993986047

И тестовые данные

In [39]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.7547995139732685

Вывод: можно использовать градиентый бустинг, но результат хотелось бы получше

### BERT

Теперь воспользуемся моделью BERT для создания эмбендингов

In [40]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert', max_length=512) 
config = transformers.BertConfig.from_pretrained('unitary/toxic-bert')
model = transformers.BertModel.from_pretrained('unitary/toxic-bert', config=config) 

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Так как вычислительные мощности компьютера не позволяют взять весь датасет, возьмём его часть

In [41]:
batch = data[:70_000]

Токениризуем предложения

In [42]:
tokenized = batch['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True)))

Находим длину самогого большого предложения и дополняем нулями те, которые меньше по длинне

In [43]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Проверяем размер

In [44]:
np.array(padded).shape

(70000, 512)

In [45]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(70000, 512)

Очищаем кэш прежде чем начать создание эмбендингов

In [46]:
torch.cuda.empty_cache()

In [47]:
from tqdm import tqdm
embeddings = []

batch_size = 50
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

for i in tqdm(range(tokenized.values.shape[0] // batch_size)):
    # преобразуем батч с токенизированными твитами в тензор 
    # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
    input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    # создаем тензор и для подготовленной маски
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
    
    # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
    # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
    with torch.no_grad():
        model.to(device)
        last_hidden_states = model(input_ids, attention_mask = attention_mask_batch)
    
    # в итоге собираем все эмбеддинги твитов в features
    embeddings.append(last_hidden_states[0][:,0,:].cpu().numpy())
    
    torch.cuda.empty_cache()

cuda:0


100%|████████████████████████████████████████████████████████████████████████████| 1400/1400 [1:27:14<00:00,  3.74s/it]


Создадим фичи и целевую переменную

In [48]:
labels = batch['toxic']
features = np.concatenate(embeddings) 

Разделяем на тренировочную и тестовую выборки

In [49]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1)

Используем CatBoost для классификации

In [50]:
%%time
model = CatBoostClassifier(
    iterations=500,
    random_seed=43,
    loss_function = 'Logloss'
)

model.fit(
    train_features, train_labels,
    eval_set=(test_features, test_labels),
    verbose=False,
)

CPU times: total: 10min 22s
Wall time: 55.2 s


Проверяем на тестовых данных

In [51]:
%%time
y_pred = model.predict(test_features)

CPU times: total: 203 ms
Wall time: 129 ms


In [52]:
f1_score(test_labels, y_pred)

0.9444444444444444

Получили отличные метрики

## Выводы

Вывод: модель BERT очень хорошо определяет токсичные комментарии и делает это сильно лучше других классификаторов, её стоит использовать в таких задачах.